In [1]:
import pandas as pd
from database.market import Market
from database.adatabase import ADatabase
from processor.processor import Processor as p
from datetime import datetime
from tqdm import tqdm
import numpy as np
from modeler_strats.universal_modeler import UniversalModeler

In [2]:
market = Market()
spec = ADatabase("btc_spec")
umod = UniversalModeler()

In [3]:
factors = [str(x) for x in range(14)]

In [6]:
training_sets = []

In [7]:
prices = pd.read_csv("./csv_files/prices/BTC-USD.csv")
prices = p.column_date_processing(prices)
prices["year"] = [x.year for x in prices["date"]]
prices["quarter"] = [x.quarter for x in prices["date"]]
ticker_data = prices
ticker_data.sort_values("date",ascending=True,inplace=True)
ticker_data["adjclose"] = [float(x) for x in ticker_data["adjclose"]]
ticker_data = ticker_data.groupby(["year","week"]).mean().reset_index()
for i in range(14):
    ticker_data[str(i)] = ticker_data["adjclose"].shift(i)
ticker_data["d1"] = ticker_data["adjclose"].pct_change(periods=1)
ticker_data["d2"] = ticker_data["d1"].pct_change(periods=1)
ticker_data["d3"] = ticker_data["d2"].pct_change(periods=1)
ticker_data["rolling10"] = ticker_data["adjclose"].rolling(window=10).mean()
ticker_data["window7"] = ticker_data["adjclose"].shift(6) - ticker_data["adjclose"].shift(1)
ticker_data["window14"] = ticker_data["adjclose"].shift(11) - ticker_data["adjclose"].shift(1)
ticker_data.dropna(inplace=True)
ticker_data["ticker"] = "BTC"
ticker_data["y"] = ticker_data["adjclose"].shift(-1)
ticker_data = ticker_data.replace([np.inf, -np.inf], np.nan).dropna()
ticker_data.dropna(inplace=True)
ticker_data = ticker_data[included_columns]
training_sets.append(ticker_data)

In [8]:
data = pd.concat(training_sets)

In [9]:
data = data.dropna()

In [10]:
data

,year,week,ticker,adjclose,y,0,1,2,3,4,5,6,7,8,9,10,11,12,13
13,2014,50,BTC,351.792712,324.602997,351.792712,375.986860,374.324855,367.477426,391.535427,342.569859,340.209289,367.316428,390.479854,355.234580,361.426570,407.692566,416.859003,314.533335
14,2014,51,BTC,324.602997,324.146572,324.602997,351.792712,375.986860,374.324855,367.477426,391.535427,342.569859,340.209289,367.316428,390.479854,355.234580,361.426570,407.692566,416.859003
15,2014,52,BTC,324.146572,293.639503,324.146572,324.602997,351.792712,375.986860,374.324855,367.477426,391.535427,342.569859,340.209289,367.316428,390.479854,355.234580,361.426570,407.692566
16,2015,1,BTC,293.639503,281.316140,293.639503,324.146572,324.602997,351.792712,375.986860,374.324855,367.477426,391.535427,342.569859,340.209289,367.316428,390.479854,355.234580,361.426570
17,2015,2,BTC,281.316140,214.185710,281.316140,293.639503,324.146572,324.602997,351.792712,375.986860,374.324855,367.477426,391.535427,342.569859,340.209289,367.316428,390.479854,355.234580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,2023,5,BTC,23273.128348,22299.014230,23273.128348,23086.560268,21754.474330,19030.571987,16866.642020,24376.750733,16782.936663,17195.546317,17073.143136,16846.483817,16374.710798,16651.560826,17542.640485,20672.421038
441,2023,6,BTC,22299.014230,23642.106306,22299.014230,23273.128348,23086.560268,21754.474330,19030.571987,16866.642020,24376.750733,16782.936663,17195.546317,17073.143136,16846.483817,16374.710798,16651.560826,17542.640485
442,2023,7,BTC,23642.106306,23905.221819,23642.106306,22299.014230,23273.128348,23086.560268,21754.474330,19030.571987,16866.642020,24376.750733,16782.936663,17195.546317,17073.143136,16846.483817,16374.710798,16651.560826
443,2023,8,BTC,23905.221819,23152.463379,23905.221819,23642.106306,22299.014230,23273.128348,23086.560268,21754.474330,19030.571987,16866.642020,24376.750733,16782.936663,17195.546317,17073.143136,16846.483817,16374.710798


In [11]:
spec.connect()
years = 4
for modeler in [umod]:
    for year in tqdm(range(2019,2023)):
        training_slice = data[(data["year"]<year) & (data["year"] >= year - years)].reset_index(drop=True)
        prediction_set = data[data["year"]==year].reset_index(drop=True)
        set_name = "sim"
        stuff = modeler.model(training_slice,prediction_set,factors)
        stuff["training_years"] = years
        spec.store(set_name,stuff)
spec.disconnect()


  0%|                                                                                                                                                              | 0/4 [00:00<?, ?it/s]

Epoch 1/10
5/5 [==============================] - 1s 2ms/step - loss: 2778466.7500 - mean_absolute_percentage_error: 18.9054
Epoch 2/10
5/5 [==============================] - 0s 1ms/step - loss: 2032780.6250 - mean_absolute_percentage_error: 14.5939
Epoch 3/10
5/5 [==============================] - 0s 1ms/step - loss: 1533492.7500 - mean_absolute_percentage_error: 12.8426
Epoch 4/10
5/5 [==============================] - 0s 1ms/step - loss: 1250886.8750 - mean_absolute_percentage_error: 12.2571
Epoch 5/10
5/5 [==============================] - 0s 1ms/step - loss: 993897.2500 - mean_absolute_percentage_error: 10.9620
Epoch 6/10
5/5 [==============================] - 0s 1ms/step - loss: 814432.2500 - mean_absolute_percentage_error: 9.6270
Epoch 7/10
5/5 [==============================] - 0s 1ms/step - loss: 648039.0000 - mean_absolute_percentage_error: 9.1722
Epoch 8/10
5/5 [==============================] - 0s 1ms/step - loss: 573821.6250 - mean_absolute_percentage_error: 8.8584
Epoch 9

 25%|█████████████████████████████████████▌                                                                                                                | 1/4 [00:06<00:18,  6.08s/it]

Epoch 1/10
5/5 [==============================] - 1s 1ms/step - loss: 6028784.0000 - mean_absolute_percentage_error: 30.3518
Epoch 2/10
5/5 [==============================] - 0s 1ms/step - loss: 4376539.0000 - mean_absolute_percentage_error: 22.9137
Epoch 3/10
5/5 [==============================] - 0s 1ms/step - loss: 2683368.2500 - mean_absolute_percentage_error: 16.2094
Epoch 4/10
5/5 [==============================] - 0s 1ms/step - loss: 2302199.7500 - mean_absolute_percentage_error: 16.7851
Epoch 5/10
5/5 [==============================] - 0s 1ms/step - loss: 1932605.3750 - mean_absolute_percentage_error: 14.5529
Epoch 6/10
5/5 [==============================] - 0s 1ms/step - loss: 1524510.3750 - mean_absolute_percentage_error: 11.5192
Epoch 7/10
5/5 [==============================] - 0s 1ms/step - loss: 1375111.1250 - mean_absolute_percentage_error: 11.0315
Epoch 8/10
5/5 [==============================] - 0s 1ms/step - loss: 1192736.1250 - mean_absolute_percentage_error: 11.3359


 50%|███████████████████████████████████████████████████████████████████████████                                                                           | 2/4 [00:12<00:12,  6.44s/it]

Epoch 1/10
5/5 [==============================] - 0s 2ms/step - loss: 62867140.0000 - mean_absolute_percentage_error: 89.1245
Epoch 2/10
5/5 [==============================] - 0s 1ms/step - loss: 15987507.0000 - mean_absolute_percentage_error: 35.5346
Epoch 3/10
5/5 [==============================] - 0s 1ms/step - loss: 8214049.0000 - mean_absolute_percentage_error: 28.3475
Epoch 4/10
5/5 [==============================] - 0s 1ms/step - loss: 11535007.0000 - mean_absolute_percentage_error: 36.3756
Epoch 5/10
5/5 [==============================] - 0s 1ms/step - loss: 8599219.0000 - mean_absolute_percentage_error: 30.0769
Epoch 6/10
5/5 [==============================] - 0s 1ms/step - loss: 5929287.0000 - mean_absolute_percentage_error: 21.2054
Epoch 7/10
5/5 [==============================] - 0s 1ms/step - loss: 5655040.0000 - mean_absolute_percentage_error: 18.4725
Epoch 8/10
5/5 [==============================] - 0s 1ms/step - loss: 5462933.5000 - mean_absolute_percentage_error: 18.13

 75%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                     | 3/4 [00:18<00:06,  6.04s/it]

Epoch 1/10
5/5 [==============================] - 0s 1ms/step - loss: 514093600.0000 - mean_absolute_percentage_error: 80.3608
Epoch 2/10
5/5 [==============================] - 0s 2ms/step - loss: 114682296.0000 - mean_absolute_percentage_error: 28.3630
Epoch 3/10
5/5 [==============================] - 0s 1ms/step - loss: 59237004.0000 - mean_absolute_percentage_error: 29.2253
Epoch 4/10
5/5 [==============================] - 0s 1ms/step - loss: 68701400.0000 - mean_absolute_percentage_error: 33.9527
Epoch 5/10
5/5 [==============================] - 0s 1ms/step - loss: 42598624.0000 - mean_absolute_percentage_error: 25.4733
Epoch 6/10
5/5 [==============================] - 0s 1ms/step - loss: 31897610.0000 - mean_absolute_percentage_error: 18.2544
Epoch 7/10
5/5 [==============================] - 0s 1ms/step - loss: 29096354.0000 - mean_absolute_percentage_error: 16.1723
Epoch 8/10
5/5 [==============================] - 0s 1ms/step - loss: 21823296.0000 - mean_absolute_percentage_error

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:23<00:00,  6.00s/it]
